In [ ]:
# https://stable-baselines3.readthedocs.io/en/master/guide/rl.html
# https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms

# 1. Import dependencies

In [2]:
!pip install stable-baselines3[extra]

     |████████████████████████████████| 161 kB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 831.4 MB 37 kB/s s eta 0:00:01     |███████████████████████████████▎| 812.4 MB 12.3 MB/s eta 0:00:02
     |████████████████████████████████| 49.9 MB 12.3 MB/s eta 0:00:01   |█████▊                          | 8.9 MB 9.3 MB/s eta 0:00:05
     |████████████████████████████████| 2.8 MB 9.5 MB/s eta 0:00:01


In [1]:
# verify pyTorch
import torch
x = torch.rand(5, 3)
print(x)
torch.cuda.is_available()

tensor([[0.7776, 0.0622, 0.8918],
        [0.3127, 0.3568, 0.0381],
        [0.8442, 0.1254, 0.2274],
        [0.0355, 0.3835, 0.8980],
        [0.4898, 0.7543, 0.2700]])


True

In [2]:
import os
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

# 2. Load Environment

In [3]:
environment_name = "CartPole-v0"

In [4]:
env = gym.make(environment_name)

In [7]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:23.0
Episode:2 Score:34.0
Episode:3 Score:13.0
Episode:4 Score:30.0
Episode:5 Score:28.0


# Understanding The Environment
https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

In [5]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

1

In [ ]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

# 3. Train an RL Model

In [6]:
log_path = os.path.join('Training','Logs')

In [7]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1,tensorboard_log=log_path) # multi-layer

Using cuda device


In [8]:
model.learn(total_timesteps=20000)

Logging to Training/Logs/PPO_4
-----------------------------
| time/              |      |
|    fps             | 491  |
|    iterations      | 1    |
|    time_elapsed    | 4    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 612         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008528819 |
|    clip_fraction        | 0.109       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | 0.00276     |
|    learning_rate        | 0.0003      |
|    loss                 | 6.47        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0164     |
|    value_loss           | 54.6        |
-----------------------------------------
---

# 4. Save and Reload Model

In [ ]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [ ]:
model.save(PPO_path)

In [ ]:
del model

In [ ]:
model = PPO.load('PPO_model', env=env)

# 4. Evaluation

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()

# 5. Test Model

In [11]:
obs = env.reset()
while True:
    action, _states = model.predict(obs) # use model to decide action
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break

info [{'TimeLimit.truncated': True, 'terminal_observation': array([-0.28821322, -0.03333642,  0.02086917, -0.00320131], dtype=float32)}]


In [12]:
env.close()

# 6. Viewing Logs in Tensorboard

In [ ]:
training_log_path = os.path.join(log_path, 'PPO_3')

In [ ]:
!tensorboard --logdir={training_log_path}

# 7. Adding a callback to the training Stage

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
import os

In [ ]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [ ]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=190, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()

# 8. Changing Policies

In [ ]:
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [ ]:
model = PPO('MlpPolicy', env, verbose = 1, policy_kwargs={'net_arch': net_arch})

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

# 9. Using an Alternate Algorithm

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

In [ ]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [ ]:
model.save(dqn_path)

In [ ]:
model = DQN.load(dqn_path, env=env)

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [ ]:
env.close()